<a href="https://colab.research.google.com/github/X-raiden/data-integration-4-scientific-dbs/blob/master/data_integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
! pip install scholarly

     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 911kB 18.3MB/s 
     |████████████████████████████████| 2.9MB 21.9MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 10.8MB 17.4MB/s 
  Created wheel for stem: filename=stem-1.8.0-cp36-none-any.whl size=436041 sha256=1483eb7a277b1a62c2cb1e76f70c13cbbb312914e371b554d61bcb1992cfae9c
  Stored in directory: /root/.cache/pip/wheels/02/3a/ee/1094b166e029353f892c0b121aa02f48aff5e658396924bc2a
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13485 sha256=b82ae1430bcd84a34e2a56b64cc0742ab48b65fe2e50e0d4c51008c36cc4c534
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
  Created wheel for bibtexparser: filename=bibtexparser-1.2.0-cp36-none-any.whl size=36712 sha256=722e4a75046a3df8f68dcade0b7a732c6b83f679b604aac7d950615c477b8c87
  Stored in directory: /root/.

In [82]:
#imports
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import json
from scholarly import scholarly, ProxyGenerator
import requests
import nltk

In [5]:
#scopus function
def scopus():
    data_details = []
    with open('web_of_science.json','r') as target:
        data = json.load(target)
    for i in range(0,6):
        resp = requests.get("http://api.elsevier.com/content/search/scopus?query=({"+data[i]['titre']+"})",headers=({"X-ELS-APIKey":"9d91974d46a4b958cafd78f34bbffcf0"}))
        #resp = requests.get("https://api.punkapi.com/v2/beers")
        data_details.append(resp.json())

    with open('scopus.json','w') as target:
        json.dump(data_details,target)
    return 1

In [6]:
#web of science function
def web_of_science(name_of_school):
    data = []
    #auteurs = []
    #keywords = []
    #it needs a geckodriver 'installed from github'
    driver = webdriver.Firefox(executable_path="/home/x-raiden/Downloads/geckodriver")
    driver.get("https://eressources.imist.ma/login?url=https://www.webofknowledge.com")
    #login to imist eressources
    login = driver.find_element_by_xpath("/html/body/div[1]/form/div[2]/input")
    password = driver.find_element_by_xpath("/html/body/div[1]/form/div[3]/input")
    #personal infos
    login.send_keys("ayoub.er-rkhis@usms.ac.ma")
    password.send_keys("@0000")
    button = driver.find_element_by_xpath("/html/body/div[1]/form/input")
    button.submit()
    #we should to wait for the page ( slow internet :'( )
    time.sleep(60)
    #entred to web_of_science
    main_selector = driver.find_element_by_xpath('//*[@id="select2-select1-container"]').click()
    search = driver.find_element_by_xpath('/html/body/span[37]/span/span[1]/input')
    search.send_keys("Enhanced")
    selector = driver.find_element_by_xpath('/html/body/span[37]/span/span[2]').click()
    search_bar = driver.find_element_by_xpath('//*[@id="value(input1)"]')
    search_bar.send_keys(name_of_school)
    button_in_web_of_science = driver.find_element_by_xpath("/html/body/form[1]/div[1]/div/div/div/table/tbody/tr/td[3]/span/span[1]/button")
    button_in_web_of_science.submit()
    #articles page
    time.sleep(30)
    driver.find_element_by_xpath('/html/body/div[1]/div[26]/div[2]/div/div/div/div[2]/div[3]/div[5]/form[2]/div/div[1]/div/span/div[2]/div[1]/div[3]/div/div[1]/div/a/value').click()
    time.sleep(16)
    for k in range (1,21):
        try:
            infos = {}
            infos['titre'] = driver.find_element_by_xpath('/html/body/div[1]/div[26]/form[3]/div/div/div/div[1]/div/div[1]/value').text
            infos['date_publication'] = driver.find_element_by_xpath('/html/body/div[1]/div[26]/form[3]/div/div/div/div[1]/div/div[3]/p[4]/value').text
            infos['type'] = driver.find_element_by_xpath('/html/body/div[1]/div[26]/form[3]/div/div/div/div[1]/div/div[3]/p[5]').text
            infos['auteurs'] = driver.find_element_by_xpath('/html/body/div[1]/div[26]/form[3]/div/div/div/div[1]/div/div[2]/p').text
            infos['auteur_mail'] = driver.find_element_by_xpath('/html/body/div[1]/div[26]/form[3]/div/div/div/div[1]/div/div[6]/p[7]/a').text
            infos['abstract'] = driver.find_element_by_xpath('/html/body/div[1]/div[26]/form[3]/div/div/div/div[1]/div/div[4]/p').text
            infos['keywords'] = driver.find_element_by_xpath('/html/body/div[1]/div[26]/form[3]/div/div/div/div[1]/div/div[5]/p[1]').text
            infos['cited'] = driver.find_element_by_xpath('/html/body/div[1]/div[26]/form[3]/div/div/div/div[2]/div[1]/div/div[6]/a/span').text
            infos['organization'] = 'Sultan Moulay Slimane University of Beni Mellal'
            data.append(infos)
            driver.find_element_by_xpath('/html/body/div[1]/div[26]/div/form/span/a[2]').click()
        except Exception as e:
            driver.find_element_by_xpath('/html/body/div[1]/div[26]/div/form/span/a[2]').click()
            pass

    with open('web_of_science.json','w') as target:
        json.dump(data,target)
    return 1


In [7]:
#google scholar function
def google_scholar():
    data_details = []
    #search_query = scholarly.search_pubs('Experimental and theoretical investigations of lignin-urea-formaldehyde wood adhesive: Density functional theory analysis')
    #scholarly.pprint(next(search_query))
    with open('web_of_science.json','r') as target:
        data = json.load(target)

    for i in range(0,6):
        name = data[i]['titre']
        search_query = scholarly.search_pubs(name)
        data_details.append(next(search_query)['bib'])
        #print(next(search_query))

    #print(data_details)
    with open('gs.json','w') as target:
        json.dump(data_details,target)
    return 1

In [8]:
#not yet completed!!!!!!!!!!!!!!!!
#science direct
def science_direct():
    data_details = []
    with open('web_of_science.json','r') as target:
        data = json.load(target)
    for i in range(0,6):
        resp = requests.get("https://api.elsevier.com/content/search/sciencedirect",headers=({"X-ELS-APIKey":"9d91974d46a4b958cafd78f34bbffcf0"}))
        #resp = requests.get("https://api.punkapi.com/v2/beers")
        #data_details.append(resp.json())
        print(resp.json())
    #with open('scopus.json','w') as target:
    #    json.dump(data_details,target)
    return 1

In [100]:
def integrate():
  matched = []
  results = []
  with open("google_scholar.json") as google:
    gs = json.load(google)

  with open("scopus.json") as scopus:
    scopus = json.load(scopus)

  with open("web_of_science.json") as wos:
    ws = json.load(wos)

  #matching between webofscience and scopus
  for i in range(len(ws)):
    for j in range(len(scopus)):
      try:
        seuil = nltk.jaccard_distance(set(ws[i]['titre']), set(scopus[j]['search-results']['entry'][0]['dc:title']))
        if seuil<=0.2:
          #print("same")
          matched.append(ws[i])
      except:
        pass

  for i in range(len(ws)):
    for j in range(len(gs)):
      try:
        seuil = nltk.jaccard_distance(set(matched[i]['titre']), set(gs[j]['title']))
        if seuil<=0.2:
          #print("same")
          results.append(ws[i])
      except:
        pass

  
  print(matched)

In [101]:
#main function
if __name__ == "__main__":
    name_of_school = "Sultan Moulay Slimane University of Beni Mellal "
    #web_of_science(name_of_school)
    #scopus()
    #google_scholar()
    #science_direct()
    integrate()

[{'titre': 'Experimental and theoretical investigations of lignin-urea-formaldehyde wood adhesive: Density functional theory analysis', 'date_publication': 'JAN 2021', 'type': 'Document Type:Article', 'auteurs': 'By:Boussetta, A (Boussetta, Abdelghani)[ 1 ] ; Benhamou, AA (Benhamou, Anass Ait)[ 1,2 ] ; Barba, FJ (Barba, Francisco J.)[ 3 ] ; Idrissi, ME (Idrissi, Mohammed EL.)[ 1 ] ; Grimi, N (Grimi, Nabil)[ 4 ] ; Moubarik, A (Moubarik, Amine)[ 1 ]', 'auteur_mail': 'amoubarik@gmail.com', 'abstract': 'In view of its chemical structure, lignin has recently become an interesting candidate for various applications. This work serves to optimize the isolation process of lignin from sugar industry by-product (bagasse can, molasses beet) applying alkali treatment. For this purpose, the effect of alkali and sulfuric acid concentrations have been studied ranging from 10% (w/v) to 20% (w/v) and 1.5 M-3 M, respectively. Moreover, the isolated lignins were characterized by different complementary an